In [1]:

import pandas as pd
import numpy as np



aisles=pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
order_products__train = pd.read_csv('order_products__train.csv')
order_products__prior = pd.read_csv('order_products__prior.csv')
sample_submission = pd.read_csv('sample_submission.csv')


In [2]:
aisles.shape
aisles.head()
aisles.nunique()




aisle_id    134
aisle       134
dtype: int64

In [3]:
departments.shape
departments.head()
departments.nunique()

department_id    21
department       21
dtype: int64

In [4]:
products.shape
products.head()
products.nunique()

product_id       49688
product_name     49688
aisle_id           134
department_id       21
dtype: int64

In [5]:
orders.shape
orders.head()
orders.nunique()


order_id                  3421083
user_id                    206209
eval_set                        3
order_number                  100
order_dow                       7
order_hour_of_day              24
days_since_prior_order         31
dtype: int64

In [6]:
order_products__prior.shape
order_products__prior.head()
order_products__prior.nunique()


order_id             3214874
product_id             49677
add_to_cart_order        145
reordered                  2
dtype: int64

In [7]:
order_products__train.shape
order_products__train.head()
order_products__train.nunique()

order_id             131209
product_id            39123
add_to_cart_order        80
reordered                 2
dtype: int64

In [8]:

df_products = products[['department_id', 'product_id']]

def rd_orders(n_clients):
    df_orders = orders
    df_orders = df_orders[['order_id','user_id']]
    return df_orders[df_orders['user_id'].isin(range(N+1))]

N=500

df_orders = rd_orders(N)
df_orders_products = order_products__prior[['order_id','product_id']]
df_orders_products = pd.merge(df_orders,df_orders_products,on = 'order_id' )
df_orders_products = pd.merge(df_orders_products,df_products,on = 'product_id' )

df_orders_products.head(10)

             


,order_id,user_id,product_id,department_id
0,2539329,1,196,7
1,2398795,1,196,7
2,473747,1,196,7
3,2254736,1,196,7
4,431534,1,196,7
5,3367565,1,196,7
6,550135,1,196,7
7,3108588,1,196,7
8,2295261,1,196,7
9,2550362,1,196,7


In [9]:


data = df_orders_products

def f_data ():
    
    L=[]

    for client in data.groupby ("user_id") :
        client_id = client [0]
        client_data = client [1]
        
        seq_client=[]
        
        for order in client_data.groupby("order_id"):
            order_id = order[0]
            order_data = order[1]
            departement_ids = list(order_data["department_id"])
            seq_client.append(departement_ids)
        L.append(seq_client)

    return L

L=f_data()

print(L)
print(len(L))

[[[7, 19, 19, 4, 16, 4, 4, 4], [7, 19, 19, 16, 13], [7, 19, 19, 14, 16], [7, 19, 17, 19, 16], [7, 19, 19, 16, 7, 16], [7, 19, 19, 19, 4, 14], [7, 16, 19, 19, 17], [7, 19, 19, 14, 16, 7, 19, 16, 16], [7, 19, 19, 16, 7, 16], [7, 19, 19, 16]], [[19, 20, 20, 7, 20, 4, 19, 4, 4, 20, 1, 1, 1], [19, 19, 7, 4, 4, 19, 16, 16, 16, 19, 20, 12, 7, 1, 4, 13], [19, 16, 4, 4, 16, 16, 16, 16, 6, 7, 4, 7, 4, 16, 13, 1], [4, 19, 13, 20, 4, 4, 19, 20, 20, 4, 4, 20, 1, 19, 16, 16, 16, 16, 16, 16, 16], [19, 19, 19, 19, 20, 4, 16, 16, 16, 16, 16, 1, 6, 19, 20], [13, 4, 4, 16, 16, 16, 16, 1, 6], [4, 4, 16, 19, 19, 20], [4, 16, 19, 19, 19, 19, 14, 14, 14, 11, 16, 16, 16], [19, 13, 19, 16, 4, 1, 19, 16, 16, 16, 16, 16, 16, 1, 19, 19, 19, 16, 4, 1, 1, 1, 1, 1, 1, 1], [19, 4, 20, 16, 20], [4, 19, 4, 20, 13, 19, 19, 19, 7, 20, 20, 4, 4], [19, 19, 20, 4, 16, 16, 16, 16, 16, 20, 16, 7, 19, 7, 4, 19, 13, 13, 3], [4, 19, 13, 19, 4, 19, 20, 7, 4, 4, 19, 16, 16, 16], [20, 19, 15, 13, 13, 19, 4, 4, 3]], [[16, 4, 19, 16,

In [10]:
import keras
from keras.preprocessing.sequence import pad_sequences


nb_clients=len(data.user_id.unique())
C = 8  # seq_catégories
T = 3  # seq_transaction
print(nb_clients)

Using TensorFlow backend.


500


In [11]:


X_1=np.zeros((500,3,8))

for i in range(len(L)):
  xi =pad_sequences(L[i], maxlen=8,dtype="int32", padding="post", truncating="pre", value=0)
  t=min(T,xi.shape[0])
  xi = xi[:t]
  X_1[i]=xi

print(X_1)


[[[ 7. 19. 19. ...  4.  4.  4.]
  [ 7. 19. 19. ...  0.  0.  0.]
  [ 7. 19. 19. ...  0.  0.  0.]]

 [[ 4. 19.  4. ...  1.  1.  1.]
  [16. 19. 20. ...  1.  4. 13.]
  [ 6.  7.  4. ... 16. 13.  1.]]

 [[ 4. 19. 16. ... 17.  4.  7.]
  [ 4. 16. 16. ...  0.  0.  0.]
  [ 4. 16. 16. ...  7.  0.  0.]]

 ...

 [[20.  1.  6. ...  1.  6. 16.]
  [ 4. 15.  7. ... 16. 13. 20.]
  [13. 16. 12. ...  4. 20.  0.]]

 [[16. 19. 20. ...  0.  0.  0.]
  [ 7.  7. 16. ...  3. 19.  7.]
  [ 4.  4.  4. ...  7. 19. 13.]]

 [[ 7.  1.  4. ...  7.  7.  7.]
  [16.  7.  7. ... 16. 11.  1.]
  [13.  4.  7. ... 19.  7.  7.]]]


In [12]:

X = X_1[:,:-1,:]
Y = X_1[:,-1,:]

print(X)
print(Y)
print(X.shape)
print(Y.shape)

[[[ 7. 19. 19. ...  4.  4.  4.]
  [ 7. 19. 19. ...  0.  0.  0.]]

 [[ 4. 19.  4. ...  1.  1.  1.]
  [16. 19. 20. ...  1.  4. 13.]]

 [[ 4. 19. 16. ... 17.  4.  7.]
  [ 4. 16. 16. ...  0.  0.  0.]]

 ...

 [[20.  1.  6. ...  1.  6. 16.]
  [ 4. 15.  7. ... 16. 13. 20.]]

 [[16. 19. 20. ...  0.  0.  0.]
  [ 7.  7. 16. ...  3. 19.  7.]]

 [[ 7.  1.  4. ...  7.  7.  7.]
  [16.  7.  7. ... 16. 11.  1.]]]
[[ 7. 19. 19. ...  0.  0.  0.]
 [ 6.  7.  4. ... 16. 13.  1.]
 [ 4. 16. 16. ...  7.  0.  0.]
 ...
 [13. 16. 12. ...  4. 20.  0.]
 [ 4.  4.  4. ...  7. 19. 13.]
 [13.  4.  7. ... 19.  7.  7.]]
(500, 2, 8)
(500, 8)


In [13]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation,Dropout
from keras.models import Model
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

def lstm(samples,num_feature):
  model = Sequential()
  model.add(LSTM(256,activation='relu', return_sequences=True, input_shape=(samples,num_feature)))
  model.add(LSTM(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(Y.shape[1], activation='softmax'))
  model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
  return model


model = lstm(X.shape[1], X.shape[2])


In [14]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2, 256)            271360    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 520       
Total params: 354,056
Trainable params: 354,056
Non-trainable params: 0
_________________________________________________________________


In [15]:
callback_1 = keras.callbacks.TensorBoard(log_dir='trainings/train-conv')
callback_2 = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=2)
callback_3 = keras.callbacks.ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)

In [16]:
model.fit(X_train, Y_train, epochs=500,batch_size=128,verbose=1, callbacks=[callback_1, callback_2, callback_3])




Epoch 1/500
375/375 [==============================] - 1s 2ms/step - loss: 105.9400 - accuracy: 0.1280

Epoch 2/500
375/375 [==============================] - 0s 157us/step - loss: 105.8356 - accuracy: 0.1867
Epoch 3/500
375/375 [==============================] - 0s 130us/step - loss: 105.7375 - accuracy: 0.1813
Epoch 4/500
375/375 [==============================] - 0s 141us/step - loss: 105.6761 - accuracy: 0.1787
Epoch 5/500

C:\Users\PANGPENGHUI\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\PANGPENGHUI\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



375/375 [==============================] - 0s 154us/step - loss: 105.6502 - accuracy: 0.1813
Epoch 6/500
375/375 [==============================] - 0s 133us/step - loss: 105.6312 - accuracy: 0.2080
Epoch 7/500
375/375 [==============================] - 0s 144us/step - loss: 105.6043 - accuracy: 0.2347
Epoch 8/500
375/375 [==============================] - 0s 146us/step - loss: 105.5712 - accuracy: 0.2907
Epoch 9/500
375/375 [==============================] - 0s 138us/step - loss: 105.5521 - accuracy: 0.2667
Epoch 10/500
375/375 [==============================] - 0s 144us/step - loss: 105.5064 - accuracy: 0.2587
Epoch 11/500
375/375 [==============================] - 0s 130us/step - loss: 105.5115 - accuracy: 0.2507
Epoch 12/500
375/375 [==============================] - 0s 146us/step - loss: 105.4502 - accuracy: 0.2827
Epoch 13/500
375/375 [==============================] - 0s 157us/step - loss: 105.4664 - accuracy: 0.2747
Epoch 14/500
375/375 [==============================] - 0s 154

375/375 [==============================] - 0s 133us/step - loss: 104.8795 - accuracy: 0.4000
Epoch 83/500
375/375 [==============================] - 0s 157us/step - loss: 104.8911 - accuracy: 0.3893
Epoch 84/500
375/375 [==============================] - 0s 178us/step - loss: 104.8766 - accuracy: 0.3893
Epoch 85/500
375/375 [==============================] - 0s 154us/step - loss: 104.8819 - accuracy: 0.3920
Epoch 86/500
375/375 [==============================] - 0s 164us/step - loss: 104.8705 - accuracy: 0.4000
Epoch 87/500
375/375 [==============================] - 0s 147us/step - loss: 104.8711 - accuracy: 0.3920
Epoch 88/500
375/375 [==============================] - 0s 113us/step - loss: 104.8678 - accuracy: 0.4000
Epoch 89/500
375/375 [==============================] - 0s 118us/step - loss: 104.8706 - accuracy: 0.3973
Epoch 90/500
375/375 [==============================] - 0s 142us/step - loss: 104.8673 - accuracy: 0.4000
Epoch 91/500
375/375 [==============================] - 0s 

375/375 [==============================] - 0s 136us/step - loss: 104.7579 - accuracy: 0.4160
Epoch 159/500
375/375 [==============================] - 0s 141us/step - loss: 104.7636 - accuracy: 0.4133
Epoch 160/500
375/375 [==============================] - 0s 120us/step - loss: 104.7487 - accuracy: 0.4133
Epoch 161/500
375/375 [==============================] - 0s 122us/step - loss: 104.7398 - accuracy: 0.4213
Epoch 162/500
375/375 [==============================] - 0s 133us/step - loss: 104.7363 - accuracy: 0.4133
Epoch 163/500
375/375 [==============================] - 0s 124us/step - loss: 104.7281 - accuracy: 0.4213
Epoch 164/500
375/375 [==============================] - 0s 121us/step - loss: 104.7389 - accuracy: 0.4213
Epoch 165/500
375/375 [==============================] - 0s 92us/step - loss: 104.7332 - accuracy: 0.4133
Epoch 166/500
375/375 [==============================] - 0s 125us/step - loss: 104.7284 - accuracy: 0.4187
Epoch 167/500
375/375 [=============================

375/375 [==============================] - 0s 145us/step - loss: 104.6298 - accuracy: 0.4507
Epoch 235/500
375/375 [==============================] - 0s 123us/step - loss: 104.6325 - accuracy: 0.4667
Epoch 236/500
375/375 [==============================] - 0s 119us/step - loss: 104.6312 - accuracy: 0.4640
Epoch 237/500
375/375 [==============================] - 0s 123us/step - loss: 104.6289 - accuracy: 0.4613
Epoch 238/500
375/375 [==============================] - 0s 119us/step - loss: 104.6270 - accuracy: 0.4613
Epoch 239/500
375/375 [==============================] - 0s 115us/step - loss: 104.6268 - accuracy: 0.4587
Epoch 240/500
375/375 [==============================] - 0s 118us/step - loss: 104.6256 - accuracy: 0.4613
Epoch 241/500
375/375 [==============================] - 0s 117us/step - loss: 104.6237 - accuracy: 0.4640
Epoch 242/500
375/375 [==============================] - 0s 111us/step - loss: 104.6247 - accuracy: 0.4613
Epoch 243/500
375/375 [============================

375/375 [==============================] - 0s 139us/step - loss: 104.5982 - accuracy: 0.4693
Epoch 311/500
375/375 [==============================] - 0s 115us/step - loss: 104.5978 - accuracy: 0.4667
Epoch 312/500
375/375 [==============================] - 0s 137us/step - loss: 104.6012 - accuracy: 0.4747
Epoch 313/500
375/375 [==============================] - 0s 103us/step - loss: 104.6056 - accuracy: 0.4747
Epoch 314/500
375/375 [==============================] - 0s 142us/step - loss: 104.6012 - accuracy: 0.4693
Epoch 315/500
375/375 [==============================] - 0s 115us/step - loss: 104.6158 - accuracy: 0.4613
Epoch 316/500
375/375 [==============================] - 0s 110us/step - loss: 104.5958 - accuracy: 0.4773
Epoch 317/500
375/375 [==============================] - 0s 116us/step - loss: 104.6187 - accuracy: 0.4507
Epoch 318/500
375/375 [==============================] - 0s 113us/step - loss: 104.6002 - accuracy: 0.4720
Epoch 319/500
375/375 [============================

375/375 [==============================] - 0s 138us/step - loss: 104.5722 - accuracy: 0.4880
Epoch 387/500
375/375 [==============================] - 0s 138us/step - loss: 104.5724 - accuracy: 0.4933
Epoch 388/500
375/375 [==============================] - 0s 138us/step - loss: 104.5734 - accuracy: 0.4880
Epoch 389/500
375/375 [==============================] - 0s 136us/step - loss: 104.5721 - accuracy: 0.4853
Epoch 390/500
375/375 [==============================] - 0s 205us/step - loss: 104.5737 - accuracy: 0.4853
Epoch 391/500
375/375 [==============================] - 0s 202us/step - loss: 104.5672 - accuracy: 0.4933
Epoch 392/500
375/375 [==============================] - 0s 197us/step - loss: 104.5683 - accuracy: 0.4880
Epoch 393/500
375/375 [==============================] - 0s 157us/step - loss: 104.5715 - accuracy: 0.4880
Epoch 394/500
375/375 [==============================] - 0s 144us/step - loss: 104.5692 - accuracy: 0.4827
Epoch 395/500
375/375 [============================

375/375 [==============================] - 0s 156us/step - loss: 104.5409 - accuracy: 0.5040
Epoch 463/500
375/375 [==============================] - 0s 161us/step - loss: 104.5476 - accuracy: 0.5013
Epoch 464/500
375/375 [==============================] - 0s 131us/step - loss: 104.5415 - accuracy: 0.5067
Epoch 465/500
375/375 [==============================] - 0s 169us/step - loss: 104.5407 - accuracy: 0.5013
Epoch 466/500
375/375 [==============================] - 0s 144us/step - loss: 104.5441 - accuracy: 0.5093
Epoch 467/500
375/375 [==============================] - 0s 181us/step - loss: 104.5371 - accuracy: 0.5093
Epoch 468/500
375/375 [==============================] - 0s 165us/step - loss: 104.5373 - accuracy: 0.5093
Epoch 469/500
375/375 [==============================] - 0s 165us/step - loss: 104.5365 - accuracy: 0.5040
Epoch 470/500
375/375 [==============================] - 0s 160us/step - loss: 104.5394 - accuracy: 0.4987
Epoch 471/500
375/375 [============================